<a href="https://colab.research.google.com/github/swiminthewind/colab/blob/main/231210-%E5%BE%97%E5%88%B0%E9%9A%8F%E6%9C%BA%E6%A0%B7%E6%9C%AC%E7%9A%84NDRE%E6%97%B6%E9%97%B4%E5%BA%8F%E5%88%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=tQAkZyj98rpm6oA5pCwe_yjM0JhT1xida-hZlAkHcq8&tc=hdyOnzZhFLlKiz86AubRqLfowEc7ZYUYYsUR0oNrUY8&cc=AcpmeooQNEAjZJe3rYUX9J2a-H6chdI0fLwqRf7fLIg

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXnBAdLRopHJ7LtP8ecNXgO4WxrNV6OKAPbS_YBSzGLjgEurofVzhVU

Successfully saved authorization token.


In [2]:
import os
import pandas as pd
import math

In [39]:
roi = ee.FeatureCollection("projects/ee-grn/assets/cd_plain_cd")
csPlus= ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")
s2sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
point = ee.FeatureCollection("projects/ee-grn/assets/rdmsample_6000")

In [4]:
timeField = 'system:time_start'
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.6

def cloud_remove(img):
    return img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)).select('.*').copyProperties(img, ['system:time_start'])

composite = s2sr.filterBounds(roi).filterDate('2022-01-01','2023-12-08').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)

def s2addVariables(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  return image.addBands(image.normalizedDifference(['B8', 'B5']).rename('NDRE')).addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1))

filteredSentinel = composite.map(s2addVariables)
print(filteredSentinel.size().getInfo())

1276


In [5]:
# 二阶谐波
dataset = filteredSentinel.select(['NDRE','t','constant'])
independents = ee.List(['constant', 't'])
dependent = ee.String('NDRE')

harmonicIndependents = ee.List(['constant', 't', 'cos1', 'sin1', 'cos2', 'sin2'])

def add_sincos(image):
  timeRadians1 = image.select('t').multiply(2 * math.pi)
  timeRadians2 = image.select('t').multiply(4 * math.pi)
  return image.addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1')).addBands(timeRadians2.cos().rename('cos2')).addBands(timeRadians2.sin().rename('sin2'))

harmonicSentinel = dataset.map(add_sincos)

harmonicTrend = harmonicSentinel.select(harmonicIndependents.add(dependent)).reduce(ee.Reducer.linearRegression(numX=harmonicIndependents.length(),numY=1))

harmonicTrendCoefficients = harmonicTrend.select('coefficients').arrayProject([0]).arrayFlatten([harmonicIndependents])

def fit(image):
  return image.addBands(image.select(harmonicIndependents).multiply(harmonicTrendCoefficients).reduce('sum').rename('fitted'))

fittedHarmonic = harmonicSentinel.map(fit)

print(fittedHarmonic.size().getInfo())

1276


In [6]:
# 5天序列NDRE
interval = 5
increment = 'day'
start = '2022-05-01'
startDate = ee.Date(start)
secondDate = startDate.advance(interval, increment).millis()
increase = secondDate.subtract(startDate.millis())
list5d = ee.List.sequence(startDate.millis(), ee.Date('2022-09-01').millis(), increase)

def add_time(date):
  return ee.Image(0).mask(0).rename('x').set('system:time_start',ee.Date(date).millis())

dailyMean =  ee.ImageCollection.fromImages(list5d.map(add_time))

def add_allbands(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  timeRadians1 = ee.Image(years).multiply(2 * math.pi)
  timeRadians2 = ee.Image(years).multiply(4 * math.pi)
  return image.addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1)).addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1')).addBands(timeRadians2.cos().rename('cos2')).addBands(timeRadians2.sin().rename('sin2'))

s2makeup = dailyMean.map(add_allbands)

def add_fittedNDRE(image):
  return image.addBands(image.select(harmonicIndependents).multiply(harmonicTrendCoefficients).reduce('sum').rename('NDRE'))

fitteds2 = s2makeup.map(add_fittedNDRE)

HANTS = fitteds2.select('NDRE')

In [7]:
# NDRE时间序列合成
image_agrgt = HANTS.toBands()
print(image_agrgt.bandNames().getInfo())

['0_NDRE', '1_NDRE', '2_NDRE', '3_NDRE', '4_NDRE', '5_NDRE', '6_NDRE', '7_NDRE', '8_NDRE', '9_NDRE', '10_NDRE', '11_NDRE', '12_NDRE', '13_NDRE', '14_NDRE', '15_NDRE', '16_NDRE', '17_NDRE', '18_NDRE', '19_NDRE', '20_NDRE', '21_NDRE', '22_NDRE', '23_NDRE', '24_NDRE']


In [8]:
image_agrgt_clip = image_agrgt.clip(roi)

In [40]:
samples_datesets=image_agrgt_clip.sampleRegions(collection=point,properties=['id'],scale=10,tileScale=8)

In [41]:
cols = ['id','0_NDRE','1_NDRE','2_NDRE','3_NDRE','4_NDRE','5_NDRE','6_NDRE','7_NDRE','8_NDRE','9_NDRE','10_NDRE','11_NDRE','12_NDRE','13_NDRE','14_NDRE','15_NDRE','16_NDRE','17_NDRE','18_NDRE','19_NDRE','20_NDRE','21_NDRE','22_NDRE','23_NDRE','24_NDRE']
inds = range(samples_datesets.size().getInfo())

In [42]:
df = pd.DataFrame(index=inds,columns=cols)
df.head()

,id,0_NDRE,1_NDRE,2_NDRE,3_NDRE,4_NDRE,5_NDRE,6_NDRE,7_NDRE,8_NDRE,...,15_NDRE,16_NDRE,17_NDRE,18_NDRE,19_NDRE,20_NDRE,21_NDRE,22_NDRE,23_NDRE,24_NDRE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
data_value = samples_datesets.reduceColumns(
    reducer = ee.Reducer.toList().repeat(26),
    selectors = cols
)
df = pd.DataFrame(data_value.getInfo()['list'],cols)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,603,604,605,606,607,608,609,610,611,612
id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0_NDRE,0.179608,0.572209,0.178418,0.466106,0.291511,0.173558,0.470657,0.252093,0.384479,0.292441,...,0.391512,0.430611,0.270506,0.377008,0.384144,0.135000,0.480417,0.271988,0.577517,0.449351
1_NDRE,0.188239,0.589907,0.185199,0.485302,0.297467,0.172853,0.483354,0.264322,0.385480,0.315954,...,0.370505,0.444584,0.283864,0.379339,0.389486,0.151814,0.506932,0.280241,0.595103,0.467718
2_NDRE,0.196622,0.604844,0.193549,0.502934,0.303661,0.172751,0.495129,0.276620,0.387839,0.339974,...,0.348968,0.457592,0.298645,0.382983,0.395071,0.171193,0.531365,0.289376,0.609352,0.484327
3_NDRE,0.204656,0.616915,0.203264,0.518819,0.310152,0.173349,0.505877,0.288815,0.391520,0.364161,...,0.327517,0.469293,0.314657,0.387985,0.400843,0.192764,0.553133,0.299366,0.619914,0.498788


In [44]:
df.to_csv('randomsample_NDRE_2022_6000.csv',encoding='utf-8')